# Assignment 3: Build a seq2seq model for machine translation.

### Name: Josh Cubero

### Task: Change LSTM model to Bidirectional LSTM Model and Translate English to Spanish

### Due Date: Wednesday, April 19th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue. 
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder **in Section 3**. Do NOT use Bi-LSTM for the decoder. But there are other codes **you need to modify** to make it work.

In [ ]:
# from keras.layers import Bidirectional, Concatenate

# encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [1]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [10]:
# e.g., filename = 'Data/deu.txt'
filename = 'spa.txt'

# e.g., n_train = 20000
n_train = 100000

In [11]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [ ]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[were young] => [somos jovenes]
[weve eaten] => [hemos comido]
[what a bore] => [que aburrimiento]
[what a dope] => [que burro eres]
[what a dope] => [que burro]
[what a heel] => [que tipo tan arrastrado]
[what a jerk] => [que pendejo]
[what a jerk] => [que imbecil]
[what a jerk] => [que cretino]
[what a life] => [que vida]


In [12]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (100000,)
Length of target_texts: (100000,)


In [13]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 35
max length of target sentences: 87


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (100000, 35)
shape of input_token_index: 27
shape of decoder_input_seq: (100000, 87)
shape of target_token_index: 29


In [15]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [ ]:
target_texts[100]

'\tsali\n'

In [16]:
decoder_input_seq[100, :]

array([10,  5,  3, 12,  9, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0], dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [17]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(100000, 35, 28)
(100000, 87, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- **You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.**

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [26]:
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model
from keras.layers import Bidirectional, Concatenate

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                    dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  [(None, 512),       583680      ['encoder_inputs[0][0]']         
 )                               (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

Print a summary and save the encoder network structure to "./encoder.pdf"

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 512),        583680      ['encoder_inputs[0][0]']         
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [27]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim * 2,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim * 2,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')
decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0][0]',        
                                 (None, 512),                     'decoder_input_h[0][0]',  

Print a summary and save the encoder network structure to "./decoder.pdf"

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0][0]',        
                                 (None, 512),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [28]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 512),        583680      ['encoder_input_x[0][0]']        
                                 (None, 512)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0]

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 512),        583680      ['encoder_input_x[0][0]']        
                                 (None, 512)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [21]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(100000, 35, 28)
shape of decoder_input_data(100000, 87, 30)
shape of decoder_target_data(100000, 87, 30)


In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=32, epochs=60, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/60
2500/2500 [==============================] - 35s 12ms/step - loss: 0.6996 - val_loss: 0.8036
Epoch 2/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.6227 - val_loss: 0.7305
Epoch 3/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5940 - val_loss: 0.6869
Epoch 4/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5722 - val_loss: 0.6480
Epoch 5/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5549 - val_loss: 0.6234
Epoch 6/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5404 - val_loss: 0.6025
Epoch 7/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5278 - val_loss: 0.5792
Epoch 8/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5176 - val_loss: 0.5651
Epoch 9/60
2500/2500 [==============================] - 29s 12ms/step - loss: 0.5084 - val_loss: 0.5515
Epoch 10/60
2500/2500 [==============================] - 29s 12m

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [30]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [31]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [32]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 [==============================] - 0s 24ms/step
-
English:        dont smoke
Spanish (true):  no fumeis
Spanish (pred):  no te puelen
1/1 [==============================] - 0s 24ms/step
-
English:        dont smoke
Spanish (true):  no fumais
Spanish (pred):  no te puelen
1/1 [==============================] - 0s 25ms/step
-
English:        dont speak
Spanish (true):  no hables
Spanish (pred):  no hables
1/1 [==============================] - 0s 25ms/step
-
English:        dont worry
Spanish (true):  no te preocupes
Spanish (pred):  no te preocupes
1/1 [==============================] - 0s 26ms/step
-
English:        dont worry
Spanish (true):  no os preocupeis
Spanish (pred):  no te preocupes
1/1 [==============================] - 0s 26ms/step
-
English:        dont worry
Spanish (true):  no se preocupen
Spanish (pred):  no te preocupes
1/1 [==============================] - 0s 24ms/step
-
English:        finish this
Spanish (true):  termine esto
Spanish (pred):  orale esto
1/1 [==

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [33]:
import numpy as np
input_sentence = 'I love you'
# Convert the sentence to lower case
input_sentence = input_sentence.lower()

# Create an empty numpy array to store the values
input_sequence = np.empty(len(input_sentence), dtype=int)

# Iterate over the characters in the sentence and get the corresponding value from the dictionary
for i, c in enumerate(input_sentence):
    if c in input_token_index:
        input_sequence[i] = input_token_index[c]

In [ ]:
print(input_sequence)
print(input_sequence.shape)

[ 5  1 11  4 23  2  1 14  4 15]
(10,)


In [34]:

input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)
input_x.shape

#translated_sentence = decode_sequence(input_x)



(10, 35, 28)

In [36]:
for seq_index in range(len(input_x)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = input_x[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + decoded_sentence)

1/1 [==============================] - 0s 26ms/step
source sentence is: i love you
translated sentence is: cospredente deberias comprar un coche



# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively? 

- In this section, you need to re-train the model we built in secton 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [37]:
# e.g., n_train = 20000
n_train2 = 30000
clean_pairs2 = clean_data(pairs)[0:n_train, :]
input_texts2 = clean_pairs[:, 0]
target_texts2 = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

In [38]:
max_encoder_seq_length = max(len(line) for line in input_texts2)
max_decoder_seq_length = max(len(line) for line in target_texts2)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 35
max length of target sentences: 87


In [47]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts2)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts2)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (100000, 35)
shape of input_token_index: 27
shape of decoder_input_seq: (100000, 87)
shape of target_token_index: 29


In [40]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoder_input_data, decoder_input_data, test_size=0.2, random_state=42)

In [42]:
split = int(0.8*len(X_train))
train, validate = X_train[:split], X_train[split:]

### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [59]:
latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                    dropout=0.25, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')


In [60]:
# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim * 2,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim * 2,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')


In [61]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [52]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(100000, 35, 28)
shape of decoder_input_data(100000, 87, 30)
shape of decoder_target_data(100000, 87, 30)


In [57]:
from keras.optimizers import Adam,SGD,RMSprop
adam = Adam(learning_rate=0.0001)
sgd = SGD(learning_rate=0.1)
rms = RMSprop(learning_rate=0.1)

In [62]:
model.compile(optimizer=adam, loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=60, validation_split=0.2)

#model.save('seq2seq2.h5')

Epoch 1/60
1250/1250 [==============================] - 19s 13ms/step - loss: 0.7792 - val_loss: 0.9548
Epoch 2/60
1250/1250 [==============================] - 13s 11ms/step - loss: 0.6808 - val_loss: 0.8753
Epoch 3/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6577 - val_loss: 0.8402
Epoch 4/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6429 - val_loss: 0.8164
Epoch 5/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6308 - val_loss: 0.7930
Epoch 6/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6199 - val_loss: 0.7776
Epoch 7/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6095 - val_loss: 0.7595
Epoch 8/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6003 - val_loss: 0.7466
Epoch 9/60
1250/1250 [==============================] - 14s 11ms/step - loss: 0.5914 - val_loss: 0.7302
Epoch 10/60
1250/1250 [==============================] - 13s 11m

ValueError: ignored

In [63]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

1/1 [==============================] - 0s 24ms/step
-
English:        dont smoke
Spanish (true):  no fumeis
Spanish (pred):  no se pueguntes
1/1 [==============================] - 0s 32ms/step
-
English:        dont smoke
Spanish (true):  no fumais
Spanish (pred):  no se pueguntes
1/1 [==============================] - 0s 32ms/step
-
English:        dont speak
Spanish (true):  no hables
Spanish (pred):  no seas para
1/1 [==============================] - 0s 25ms/step
-
English:        dont worry
Spanish (true):  no te preocupes
Spanish (pred):  no te preguntes
1/1 [==============================] - 0s 26ms/step
-
English:        dont worry
Spanish (true):  no os preocupeis
Spanish (pred):  no te preguntes
1/1 [==============================] - 0s 24ms/step
-
English:        dont worry
Spanish (true):  no se preocupen
Spanish (pred):  no te preguntes
1/1 [==============================] - 0s 23ms/step
-
English:        finish this
Spanish (true):  termine esto
Spanish (pred):  le pada e

In [ ]:
type(decoded_sentence)

str

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.
2. A reasonable should be 0.1-0.3. The higher, the better.

In [71]:

cleaned_data = clean_data(pairs)[n_train: n_train + 100]
input_test_texts, target_test_texts = cleaned_data[:, 0], cleaned_data[:, 1]

In [72]:
input_sequence = np.empty(len(input_test_texts), dtype=int)

# Iterate over the characters in the sentence and get the corresponding value from the dictionary
for i, c in enumerate(input_test_texts):
    if c in input_token_index:
        input_sequence[i] = input_token_index[c]

In [90]:
cleaned_data = clean_data(pairs)[n_train: n_train + 100]
input_test_texts, target_test_texts = cleaned_data[:, 0], cleaned_data[:, 1]

input_sequences = np.zeros((len(input_test_texts), max_encoder_seq_length), dtype="int")
for i, input_text in enumerate(input_test_texts):
    input_text = input_text[:max_encoder_seq_length] # truncate input_text if it is longer than max_encoder_seq_length
    for t, char in enumerate(input_text):
        if char in input_token_index:
            input_sequences[i, t] = input_token_index[char]


In [91]:
input_xs = onehot_encode(input_sequences, max_encoder_seq_length, num_encoder_tokens)
input_xs.shape


(100, 35, 28)

In [92]:
from nltk.translate.bleu_score import sentence_bleu

score = 0
for i, input_sentence in enumerate(input_xs):
    # Take one sequence (part of the testing set) 
    # for trying out decoding.
    input_seq = input_xs[i:i+1]
    decoded_sentence = decode_sequence(input_seq)
    print(decoded_sentence)
    reference = [decoded_sentence.split()]  # wrap in list
    print(reference)
    candidate = target_test_texts[i].split()  # convert to string and split into words
    print(candidate)
    s = sentence_bleu(reference, candidate)
    score += s
    print(score)
print("Average BLEU score {0:.3f}".format(score / len(input_xs)))

1/1 [==============================] - 0s 33ms/step
tom y mary son anas manas de mines

[['tom', 'y', 'mary', 'son', 'anas', 'manas', 'de', 'mines']]
['tom', 'y', 'mary', 'estan', 'de', 'luna', 'de', 'miel']
4.797597231912944e-78
1/1 [==============================] - 0s 26ms/step
tom y mary son anas manas de mines

[['tom', 'y', 'mary', 'son', 'anas', 'manas', 'de', 'mines']]
['tom', 'y', 'mary', 'estan', 'en', 'su', 'luna', 'de', 'miel']
9.13271570318253e-78
1/1 [==============================] - 0s 26ms/step
tom y mary se estan porando do pasa

[['tom', 'y', 'mary', 'se', 'estan', 'porando', 'do', 'pasa']]
['tom', 'y', 'mary', 'juegan', 'a', 'hundir', 'la', 'nave']
1.3597382999215428e-77
1/1 [==============================] - 0s 25ms/step
tom y mary son anaran a la perra

[['tom', 'y', 'mary', 'son', 'anaran', 'a', 'la', 'perra']]
['tom', 'y', 'mary', 'estan', 'orgullosos', 'de', 'su', 'hijo']
1.8062050295248326e-77
1/1 [==============================] - 0s 25ms/step
tom y mary son 